<a href="https://colab.research.google.com/github/DinaTTomoum/Thesis2021/blob/main/Trump2016.LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import spacy
import re
import string
import pandas as pd
import numpy as np
! pip install --upgrade gensim
import gensim
from gensim import corpora
!python -m pip install -U pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import json
%matplotlib inline
# Import the libraries
import tweepy
from textblob import TextBlob
from nltk.corpus import stopwords
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (3.8.3)
Requirement already up-to-date: pyLDAvis in /usr/local/lib/python3.7/dist-packages (3.2.2)


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [ ]:
def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

def remove_users(tweet):
    '''Takes a string and removes retweet and @user information'''
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    return tweet

<input>:10: DeprecationWarning: invalid escape sequence \s
<input>:10: DeprecationWarning: invalid escape sequence \s
<input>:10: DeprecationWarning: invalid escape sequence \s
<ipython-input-5-869d16050efb>:10: DeprecationWarning: invalid escape sequence \s
  tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet


In [73]:
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'

# cleaning master function
def clean_tweet(tweet, bigrams=False):
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = tweet.lower() # lower case
    tweet = re.sub('['+my_punctuation + ']+', ' ', tweet) # strip punctuation
    tweet = re.sub('([0-9]+)', '', tweet) # remove numbers
    tweet = re.sub(r'[^\x00-\x7F]+','', tweet)
    tweet_token_list = [word for word in tweet.split(' ')
                            if word not in my_stopwords] # remove stopwords

    if bigrams:
        tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
                                            for i in range(len(tweet_token_list)-1)]
    tweet = ' '.join(tweet_token_list)
    return tweet

In [90]:
from google.colab import files
uploaded = files.upload()

Saving Trump2016.csv to Trump2016 (1).csv


In [91]:
df = pd.read_csv('Trump2016 (1).csv', encoding= 'unicode_escape')
df.head()

,handle,Election,text,is_retweet,original_author,date,time,in_reply_to_screen_name,is_quote_status,lang,retweet_count,favorite_count,place_full_name,place_name,place_country,source_url
0,realDonaldTrump,2016,Join me for a 3pm rally - tomorrow at the Mid-...,False,NaN,9/27/2016,22:13:24,NaN,False,en,2181,6172,NaN,NaN,NaN,http://twitter.com/download/iphone
1,realDonaldTrump,2016,"Once again, we will have a government of, by a...",False,NaN,9/27/2016,21:08:22,NaN,False,en,4132,11239,NaN,NaN,NaN,http://twitter.com/download/iphone
2,realDonaldTrump,2016,"On National #VoterRegistrationDay, make sure y...",True,GOP,9/27/2016,20:31:14,NaN,False,en,2953,6963,NaN,NaN,NaN,http://twitter.com/download/iphone
3,realDonaldTrump,2016,Hillary Clinton's Campaign Continues To Make F...,False,NaN,9/27/2016,20:14:33,NaN,False,en,3833,9840,NaN,NaN,NaN,http://twitter.com/download/iphone
4,realDonaldTrump,2016,"'CNBC, Time magazine online polls say Donald T...",False,NaN,9/27/2016,20:06:25,NaN,False,en,4236,10948,NaN,NaN,NaN,http://twitter.com/download/iphone


In [92]:
df['clean_tweet'] = df.text.apply(clean_tweet)
df.head()

,handle,Election,text,is_retweet,original_author,date,time,in_reply_to_screen_name,is_quote_status,lang,retweet_count,favorite_count,place_full_name,place_name,place_country,source_url,clean_tweet
0,realDonaldTrump,2016,Join me for a 3pm rally - tomorrow at the Mid-...,False,NaN,9/27/2016,22:13:24,NaN,False,en,2181,6172,NaN,NaN,NaN,http://twitter.com/download/iphone,join pm rally tomorrow mid america center co...
1,realDonaldTrump,2016,"Once again, we will have a government of, by a...",False,NaN,9/27/2016,21:08:22,NaN,False,en,4132,11239,NaN,NaN,NaN,http://twitter.com/download/iphone,government people join movement today
2,realDonaldTrump,2016,"On National #VoterRegistrationDay, make sure y...",True,GOP,9/27/2016,20:31:14,NaN,False,en,2953,6963,NaN,NaN,NaN,http://twitter.com/download/iphone,national #voterregistrationday make sure regi...
3,realDonaldTrump,2016,Hillary Clinton's Campaign Continues To Make F...,False,NaN,9/27/2016,20:14:33,NaN,False,en,3833,9840,NaN,NaN,NaN,http://twitter.com/download/iphone,hillary clinton campaign continues make false ...
4,realDonaldTrump,2016,"'CNBC, Time magazine online polls say Donald T...",False,NaN,9/27/2016,20:06:25,NaN,False,en,4236,10948,NaN,NaN,NaN,http://twitter.com/download/iphone,cnbc time magazine online polls say donald t...


In [93]:
from sklearn.feature_extraction.text import CountVectorizer

# the vectorizer object will be used to transform text to vector form
vectorizer = CountVectorizer(max_df=0.9, min_df=25, token_pattern='\w+|\$[\d\.]+|\S+')

# apply transformation
tf = vectorizer.fit_transform(df['clean_tweet']).toarray()

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names()

<input>:4: DeprecationWarning: invalid escape sequence \w
<input>:4: DeprecationWarning: invalid escape sequence \w
<input>:4: DeprecationWarning: invalid escape sequence \w
<input>:4: DeprecationWarning: invalid escape sequence \w
<input>:4: DeprecationWarning: invalid escape sequence \w
<input>:4: DeprecationWarning: invalid escape sequence \w
<input>:4: DeprecationWarning: invalid escape sequence \w
<ipython-input-93-a8b6bae9794c>:4: DeprecationWarning: invalid escape sequence \w
  vectorizer = CountVectorizer(max_df=0.9, min_df=25, token_pattern='\w+|\$[\d\.]+|\S+')


In [94]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 10

model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

In [95]:
model.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [96]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

In [97]:
no_top_words = 10
display_topics(model, tf_feature_names, no_top_words)

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights,Topic 5 words,Topic 5 weights,Topic 6 words,Topic 6 weights,Topic 7 words,Topic 7 weights,Topic 8 words,Topic 8 weights,Topic 9 words,Topic 9 weights
0,tonight,90.0,#trump,360.1,great,124.6,trump,72.1,hillary,360.3,people,217.8,cruz,202.3,great,120.9,new,190.1,trump,306.1
1,enjoy,89.1,thank,352.9,big,102.8,much,59.0,clinton,207.1,like,91.7,america,179.3,thank,108.2,amp,100.7,president,87.8
2,interviewed,64.1,#makeamericagreatagain,240.1,last,82.3,watch,56.5,crooked,186.1,many,83.5,make,150.8,see,69.1,poll,70.4,bernie,71.5
3,media,56.6,join,105.1,amazing,75.1,one,54.6,would,53.1,get,64.4,ted,132.1,soon,55.1,great,58.4,go,57.7
4,morning,54.8,#votetrump,71.1,night,74.1,time,51.5,bad,51.9,jobs,58.9,great,104.4,support,50.6,hampshire,48.1,win,52.9
5,dishonest,37.3,tomorrow,57.8,vote,63.6,ever,42.1,never,47.4,going,47.0,jeb,49.1,national,44.1,york,46.1,donald,51.5
6,show,36.2,pm,55.1,carolina,63.1,look,38.7,years,47.1,way,46.9,rubio,45.3,job,43.1,thank,41.9,sanders,41.1
7,rubio,35.9,video,35.6,day,54.4,better,36.1,u,43.3,republican,46.6,lyin,41.1,back,43.0,america,35.9,right,36.1
8,ads,34.1,#trumptrain,33.6,crowd,53.2,totally,34.3,women,39.4,bad,33.8,#trumppence,29.1,speech,42.6,stop,35.1,amp,35.9
9,failing,33.8,us,32.8,today,50.8,great,33.7,obama,39.1,good,31.5,bush,27.6,#americafirst,39.9,together,34.5,says,34.4
